https://oauth.vk.com/authorize?client_id=7765951&scope=14&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.16&response_type=token

In [8]:
access_token = 'b369c70bf27c0cff009eaaadc1fe3eb3053cde0614297edc9231514988c697471429e3751089737939a86'
version = '5.130'

In [5]:
import requests
import json
import time
from tqdm.notebook import tqdm
from datetime import datetime

In [6]:
def dictToListOfStrings(dictionary):
    listOfStrings=[]
    for key in dictionary:
        listOfStrings.append(str(key) + '=' + str(dictionary[key]))
    return listOfStrings

def vkMethod(method_name, parameters, access_token, version):
    url = 'https://api.vk.com/method/{method_name}?{parameters}&access_token={access_token}&v={version}'
    url = url.format(method_name=method_name,
               parameters='&'.join(dictToListOfStrings(parameters)),
               access_token=access_token,
               version=version)
#     print(url)
    time.sleep(0.3)
    response = requests.get(url).json()
    return response

def createBatches(someList, batchSize=25):
    batches = []
    for i in range(0, len(someList), batchSize):
        batches.append(someList[i: i + batchSize])
    return batches

def createBatchedExecuteCode(community_id, batchSize):
    apiRequests = []
    for i in range(0, getCommunityWallsCount(community_id), 100):
        params = {'owner_id': community_id, 'count':100, 'offset':i}
        request = 'API.wall.get(' + json.dumps(params) + ')'
        apiRequests.append(request)
    # ПРИ 15 уже падает
    return createBatches(apiRequests, batchSize)

def createBatchedExecuteCode(method, params, batchSize, totalCount = None):
    apiRequests = []
    totalCount = getObjectCount(method, params) if totalCount is None else totalCount
    for i in range(0, totalCount, 100):
        params_i = params.copy()
        params_i['offset'] = i
        request = 'API.{method}({params})'.format(method = method, params = params_i)
        apiRequests.append(request)
    return createBatches(apiRequests, batchSize)

def getObjectCount(method, params):
    response = vkMethod(method, params, access_token, version)
    return response['response']['count']

fieldsToKeep = ['id', 'owner_id', 'date', 'text', 'marked_as_ads', 'comments', 'likes', 'reposts', 'views', 'thread', 'post_id']
def deleteFromDict(dictionary, fieldsToKeep):
    new = dictionary.copy()
    for field in list(new.keys() - fieldsToKeep):
        new.pop(field, None)
    return new

# new_Walls = list(map(lambda item: deleteFromDict(item, fieldsToKeep), allWalls))
# params = {'owner_id': -193013221, 'count':100, 'offset':i}

In [3]:
import json
with open('config.json', 'r') as f:
    properties = json.loads(f.read())

communities_params = properties['communities_params']
filters = properties['filters']

In [7]:
date_time_obj = datetime.strptime('2019-11-01', '%Y-%m-%d')
first_news = int(time.mktime(date_time_obj.timetuple()))


def rawPosts(community_params):
    allWalls = []
    params = {'owner_id': community_params['community_id'], 'count':100}
    codeInBatches = createBatchedExecuteCode('wall.get', params, batchSize = 12)
    for code in tqdm(codeInBatches):
        executeRequests = ', '.join(code)
        executeCode = 'return[{requests}];'.format(requests = executeRequests)
        response = vkMethod('execute', {'code': executeCode}, access_token, version)['response']
        
        for batch in response:
            allWalls += batch['items']
        
        if allWalls[-1]['date'] < first_news:
            break
    with open('posts/rawPosts_{name}.json'.format(name=community_params['name']), 'w+') as f:
        clean = list(map(lambda item: deleteFromDict(item, fieldsToKeep), allWalls))
        if community_params['tag_filter']:
            clean = list(
                filter(
                    lambda item: any(filter_ in item['text'].lower() for filter_ in filters),
                    clean
                )
            )
        json.dump(clean, f)
    return len(codeInBatches)

In [279]:
rawPosts(communities_params[0])
rawPosts(communities_params[1])
rawPosts(communities_params[2])
rawPosts(communities_params[3])
rawPosts(communities_params[4])

4

In [9]:
rawPosts(communities_params[5])
rawPosts(communities_params[6])
rawPosts(communities_params[7])
rawPosts(communities_params[8])

113

In [271]:
name = communities_params[0]['name']
with open('rawPosts_{name}.json'.format(name=name)) as json_file:
    data = json.load(json_file)
    print(len(data))
#     break

3700


In [272]:
name = communities_params[1]['name']
with open('rawPosts_{name}.json'.format(name=name)) as json_file:
    data = json.load(json_file)
    print(len(data))
#     break

9485


In [273]:
name = communities_params[2]['name']
with open('rawPosts_{name}.json'.format(name=name)) as json_file:
    data = json.load(json_file)
    print(len(data))
#     break

1077


In [287]:
name = communities_params[3]['name']
with open('posts/rawPosts_{name}.json'.format(name=name)) as json_file:
    data = json.load(json_file)
    print(len(data))
#     break

3327


In [288]:
name = communities_params[4]['name']
with open('posts/rawPosts_{name}.json'.format(name=name)) as json_file:
    data = json.load(json_file)
    print(len(data))
#     break

1583


In [274]:
name = communities_params[0]['name']
all_comments = 0
with open('rawPosts_{name}.json'.format(name=name)) as json_file:
    data = json.load(json_file)
    for post in data:
        all_comments += post['comments']['count']
all_comments

199208

In [275]:
name = communities_params[1]['name']
all_comments = 0
with open('rawPosts_{name}.json'.format(name=name)) as json_file:
    data = json.load(json_file)
    for post in data:
        all_comments += post['comments']['count']
all_comments

222001

In [276]:
name = communities_params[2]['name']
all_comments = 0
with open('rawPosts_{name}.json'.format(name=name)) as json_file:
    data = json.load(json_file)
    for post in data:
        all_comments += post['comments']['count']
all_comments

33283

In [289]:
name = communities_params[3]['name']
all_comments = 0
with open('posts/rawPosts_{name}.json'.format(name=name)) as json_file:
    data = json.load(json_file)
    for post in data:
        all_comments += post['comments']['count']
all_comments

205803

In [290]:
name = communities_params[4]['name']
all_comments = 0
with open('posts/rawPosts_{name}.json'.format(name=name)) as json_file:
    data = json.load(json_file)
    for post in data:
        all_comments += post['comments']['count']
all_comments

89388

In [10]:
def getPostIdsAndCounts(community_params):
    ids = []
    with open('posts/rawPosts_{name}.json'.format(name=community_params['name'])) as json_file:
        data = json.load(json_file)
        for post in data:
            ids.append((post['id'], post['comments']['count']))
    return ids

In [11]:
def rawComments(community_params):
    allComments = []
    post_ids = getPostIdsAndCounts(community_params)
    for post_id, count in tqdm(post_ids):
        params = {'owner_id': community_params['community_id'], 'post_id':post_id, 'count':100}
        codeInBatches = createBatchedExecuteCode('wall.getComments', params, batchSize = 25, totalCount=count)
        for code in codeInBatches:
            executeRequests = ', '.join(code)
            executeCode = 'return[{requests}];'.format(requests = executeRequests)
            response = vkMethod('execute', {'code': executeCode}, access_token, version)
            try:
                response = response['response']
            except:
                print(response)
            for batch in response:
                comments = batch['items'].copy()
                for comment in comments:
                    comment['post_id'] = post_id
#                 print(comment)
#                 comment = list(map(lambda item: item.update({'post_id':post_id}), batch['items']))
#                 comment['post_id'] = post_id
                allComments += comments
    with open('comments/rawComments_{name}.json'.format(name=community_params['name']), 'w+') as f:
#         clean = list(map(lambda item: deleteFromDict(item, fieldsToKeep), allComments))
#         json.dump(clean, f)
        json.dump(allComments, f)
    return len(codeInBatches)

In [391]:
for community_params in communities_params:
    rawComments(community_params)

In [12]:
rawComments(communities_params[5])
rawComments(communities_params[6])
rawComments(communities_params[7])
rawComments(communities_params[8])

ConnectionError: HTTPSConnectionPool(host='api.vk.com', port=443): Max retries exceeded with url: /method/execute?code=return%5BAPI.wall.getComments(%7B'owner_id':%20-29534144,%20'post_id':%2014305592,%20'count':%20100,%20'offset':%200%7D),%20API.wall.getComments(%7B'owner_id':%20-29534144,%20'post_id':%2014305592,%20'count':%20100,%20'offset':%20100%7D)%5D;&access_token=b369c70bf27c0cff009eaaadc1fe3eb3053cde0614297edc9231514988c697471429e3751089737939a86&v=5.130 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff58b1a55e0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [13]:
rawComments(communities_params[6])

1

In [14]:
rawComments(communities_params[7])
rawComments(communities_params[8])

1

In [15]:
import os

filelist = os.listdir('comments/')
sum_ = 0
for file in filelist:
    with open('comments/' + file) as json_file:
        data = json.load(json_file)
        print(len(data))
        sum_ += len(data)
sum_

9493
23655
48975
102134
12723
56558
36922
73957
89100


453517

In [316]:
# filelist
with open('posts/' + 'rawPosts_ВОЗ.json') as json_file:
    data = json.load(json_file)
    print(data[0])

{'id': 63005, 'owner_id': -193849773, 'date': 1616439600, 'marked_as_ads': 0, 'text': 'Представитель Всемирной организации здравоохранения (ВОЗ) в России Мелита Вуйнович порекомендовала россиянам для поддержания здоровья весной есть побольше овощей и фруктов, заниматься физической активностью и не забывать про гигиену и ношение масок. \n \n«Во-первых, надо ответственно себя вести по отношению к собственному здоровью. Мы сами можем очень много сделать для своего здоровья без больших затрат: это здоровая еда — фрукты, овощи пять раз в день, меньше сахара, меньше соли, даже в переработанных продуктах, прогулки, — заявила Вуйнович в интервью ТАСС, отвечая на соответствующий вопрос. — При этом пользоваться маской, обращать внимание на гигиену». \nОна указала, что дополнительно витамины можно принимать только по назначению врача. «Очень важно, чтобы [в организме] было достаточно витамина D, для этого нужно находиться на солнце», — добавила представитель организации. \n«Самое главное, беречь 

In [ ]:
params = {'owner_id': -193849773, 'post_id':63005, 'comment_id':63028, 'count':1}
vkMethod('wall.getComments', params, access_token, version)

In [390]:
# filelist
tmp = []
with open('comments/' + 'rawComments_ВОЗ.json') as json_file:
    data = json.load(json_file)
#     print(data[0])
    for comment in data:
        try:
            tmp.append(comment['post_id'])
        except:
            print(comment)

In [318]:
params = {'owner_id': -193849773, 'post_id':63005, 'count':1}
vkMethod('wall.getComments', params, access_token, version)

{'response': {'count': 34,
  'items': [{'id': 63028,
    'from_id': 49103030,
    'post_id': 63005,
    'owner_id': -193849773,
    'parents_stack': [],
    'date': 1616444956,
    'text': 'Вы погоду видели? На улице гололёд и заморозки, какие нахрен 10 тысяч шагов? В такую погоду лучше по улице ходить как можно меньше, конечности целее будут.',
    'attachments': [{'type': 'photo',
      'photo': {'album_id': -8,
       'date': 1616445206,
       'id': 457251717,
       'owner_id': -193849773,
       'has_tags': False,
       'access_key': '6bede4dfb3e5bee2ee',
       'sizes': [{'height': 98,
         'url': 'https://sun1-29.userapi.com/impf/W5rLuQpv_P-qQluO0xN4h4Za4OIHOSCtOKx9MQ/CPm2Rh6zQFc.jpg?size=130x98&quality=96&sign=2227e7aea6256e5dbdc731e4b23d055d&c_uniq_tag=NUtiJHX6_IT4YOlL-EKiqe92jKj9Xi9dOLI9dZvGy4M&type=album',
         'type': 'm',
         'width': 130},
        {'height': 98,
         'url': 'https://sun1-29.userapi.com/impf/W5rLuQpv_P-qQluO0xN4h4Za4OIHOSCtOKx9MQ/CPm2Rh6

In [319]:
params = {'owner_id': -193849773, 'post_id':63005, 'comment_id':63028, 'count':1}
vkMethod('wall.getComments', params, access_token, version)

{'response': {'count': 3,
  'items': [{'id': 63063,
    'from_id': 2058732,
    'post_id': 63005,
    'owner_id': -193849773,
    'parents_stack': [63028],
    'date': 1616478332,
    'text': 'Я гуляю в любую погоду. А тренировать реакцию и эквилибристику на таких дорогах самое то. Мышцы икр ног хорошо тренируются в такую погоду.'}],
  'current_level_count': 3,
  'can_post': True,
  'show_reply_button': True,
  'groups_can_post': True}}

In [16]:
def extractCommentIdsAndPostIdAndCount(community_params):
    ids = []
    with open('comments/rawComments_{name}.json'.format(name=community_params['name'])) as json_file:
        data = json.load(json_file)
        for comment in data:
            ids.append((comment['id'], comment['post_id'], comment['thread']['count']))
    return ids

def getAnswerTexts(community_params):
    allComments = []
    for commentId, postId, count in tqdm(extractCommentIdsAndPostIdAndCount(community_params)):
        if count > 0:
            params = {'owner_id': community_params['community_id'], 'post_id':postId, 'comment_id':commentId, 'count':100}
            codeInBatches = createBatchedExecuteCode('wall.getComments', params, batchSize = 25, totalCount = count)
            for code in codeInBatches:
                executeRequests = ', '.join(code)
                executeCode = 'return[{requests}];'.format(requests = executeRequests)
                response = vkMethod('execute', {'code': executeCode}, access_token, version)
                try:
                    response = response['response']
                except:
                    print(response)
                for batch in response:
                    allComments += batch['items']
    with open('answers/rawAnswers_{name}.json'.format(name=community_params['name']), 'w+') as f:
        clean = list(map(lambda item: deleteFromDict(item, fieldsToKeep), allComments))
        json.dump(clean, f)

In [ ]:
getAnswerTexts(communities_params[1])

In [ ]:
getAnswerTexts(communities_params[0])

In [ ]:
getAnswerTexts(communities_params[2])

In [ ]:
getAnswerTexts(communities_params[3])

In [ ]:
getAnswerTexts(communities_params[4])

In [17]:
getAnswerTexts(communities_params[5])

In [ ]:
getAnswerTexts(communities_params[6])

In [ ]:
getAnswerTexts(communities_params[7])

In [ ]:
getAnswerTexts(communities_params[8])

In [311]:
'comments/' +filelist[0]

'comments/rawComments_ВОЗ.json'

In [306]:
with open('comments/rawComments_{name}.json'.format(name=communities_params[0]['name'])) as json_file:
    data = json.load(json_file)
    print(len(data))

73951


In [423]:
with open('answers/rawAnswers_{name}.json'.format(name=communities_params[2]['name'])) as json_file:
    data = json.load(json_file)
    print(len(data))
    print(data[5])

23799
{'id': 63141, 'post_id': 63005, 'owner_id': -193849773, 'date': 1616529773, 'text': 'Есть способ проще'}


In [ ]:
def extractCommentIds(community_params):
    ids = []
    with open('comments/rawComments_{name}.json'.format(name=community_params['name'])) as json_file:
        data = json.load(json_file)
        for comment in data:
            ids.append(comment['id'])
    return ids

def getAnswerTexts(community_params):
    allComments = []
    params = {'owner_id': community_params['community_id'], 'post_id':post_id, 'comment_id':383258, 'count':100}
    codeInBatches = createBatchedExecuteCode('wall.getComments', params, batchSize = 25)
    for code in tdqm(codeInBatches):
        executeRequests = ', '.join(code)
        executeCode = 'return[{requests}];'.format(requests = executeRequests)
        response = vkMethod('execute', {'code': executeCode}, access_token, version)
        try:
            response = response['response']
        except:
            print(response)
        for batch in response:
            allComments += batch['items']
    with open('comments/rawComments_{name}.json'.format(name=community_params['name']), 'w+') as f:
        clean = list(map(lambda item: deleteFromDict(item, fieldsToKeep), allComments))
        json.dump(clean, f)

In [ ]:
rawComments(communities_params[1])

In [137]:
def rawComment(community_params, post_id):
    allComments = []
    params = {'owner_id': community_params['community_id'], 'post_id':post_id, 'comment_id':383258, 'count':100}
    codeInBatches = createBatchedExecuteCode('wall.getComments', params, batchSize = 50)
    for code in codeInBatches:
        executeRequests = ', '.join(code)
        executeCode = 'return[{requests}];'.format(requests = executeRequests)
        response = vkMethod('execute', {'code': executeCode}, access_token, version)
        try:
            response = response['response']
        except:
            print(response)
        for batch in response:
            allComments += batch['items']
    with open('tmp1.json', 'w+') as f:
        clean = list(map(lambda item: deleteFromDict(item, fieldsToKeep), allComments))
        json.dump(clean, f)
    return len(codeInBatches)

rawComment(communities_params[0], 383227)

1

In [ ]:
params = {'owner_id': community_params['community_id'], 'post_id':post_id, 'comment_id':63028, 'count':100}
vkMethod('wall.getComments', params, access_token, version)

In [138]:
with open('tmp1.json') as json_file:
    data = json.load(json_file)
    print(data)

[{'id': 383276, 'owner_id': -193013221, 'date': 1616422537, 'text': '[id575649485|Ирина], то-то люди в очередях от недель до месяцев. А на Ростовскую область очередная партия всего 9100 доз.'}, {'id': 383287, 'owner_id': -193013221, 'date': 1616422936, 'text': '[id77827202|Дмитрий], вообще дисбаланс довольно большой по стране. У меня в городе вакцины хватает, людей желающих мало уже. А где-то привиться хотят, а не хватает вакцины.'}, {'id': 383289, 'owner_id': -193013221, 'date': 1616423040, 'text': '[id183537220|Алексей], я сам привился из самой первой партии, которая для врачей была. Но те отказались, из 20 тысяч половина ушла гражданам.)) Пусть будет вакцина, а желающие найдутся.'}, {'id': 383298, 'owner_id': -193013221, 'date': 1616423447, 'text': '[id77827202|Дмитрий], ну вот в Зеленограде немного по другому) человек из 30,например, на работе моей, только один привитый. Боятся чего-то, сами не знают чего. Но пример известных людей был бы очень кстати, может  последуют.'}, {'id': 3

In [ ]:
def rawComment(community_params, post_id):
    allComments = []
    params = {'owner_id': community_params['community_id'], 'extended':1, 'post_id':post_id, 'count':100}
    codeInBatches = createBatchedExecuteCode('wall.getComments', params, batchSize = 50)
    for code in codeInBatches:
        executeRequests = ', '.join(code)
        executeCode = 'return[{requests}];'.format(requests = executeRequests)
        response = vkMethod('execute', {'code': executeCode}, access_token, version)
        try:
            response = response['response']
        except:
            print(response)
        for batch in response:
            allComments += batch['items']
    with open('tmp.json', 'w+') as f:
        clean = list(map(lambda item: deleteFromDict(item, fieldsToKeep), allComments))
        json.dump(clean, f)
    return len(codeInBatches)

rawComment(communities_params[0], 383227)

In [136]:
with open('tmp.json') as json_file:
    data = json.load(json_file)
    print(data[6])

{'id': 383258, 'owner_id': -193013221, 'date': 1616421329, 'text': 'В ход пошла " тяжелая артиллерия" ,вакцинация слабо продвигается'}


In [115]:
### name = communities_params[0]['name']
with open('rawComments_{name}.json'.format(name=name)) as json_file:
    data = json.load(json_file)
    print(len(data))

65303


In [121]:
### name = communities_params[0]['name']
with open('rawComments_{name}.json'.format(name=name)) as json_file:
    data = json.load(json_file)
    print(data[10])

{'id': 383291, 'owner_id': -193013221, 'date': 1616423155, 'text': 'Молодец. Больше людей после этого пойдут прививаться. Не понимаю, чего он раньше не укололся.'}


In [51]:
response = vkMethod('wall.getComments', {'owner_id': -193013221, 'post_id':383227, 'count':1,}, access_token, version)

In [52]:
response

{'response': {'count': 32,
  'items': [{'id': 383229,
    'from_id': 77827202,
    'post_id': 383227,
    'owner_id': -193013221,
    'parents_stack': [],
    'date': 1616419685,
    'text': 'Ну и норм. Как раз перед стартом массового производства. Видно, что заранее спланировали умные головы.)',
    'thread': {'count': 0,
     'items': [],
     'can_post': False,
     'show_reply_button': False}}],
  'current_level_count': 19,
  'can_post': False,
  'show_reply_button': False}}

In [5]:
# TODO: number of posts

allWalls = []
for tmp_ in createBatchedExecuteCode():
    executeRequests = ', '.join(tmp_)
    executeCode = 'return[{requests}];'.format(requests = executeRequests)
    response = vkMethod('execute', {'code': executeCode}, access_token, version)['response']
    for batch in response:
        allWalls += batch['items']

In [65]:
with open('rawPosts.json', 'w') as f:
    json.dump(allWalls, f)

In [16]:
fieldsToKeep = ['id', 'owner_id', 'date', 'text', 'marked_as_ads', 'comments', 'likes', 'reposts', 'views']
def deleteFromDict(dictionary, fieldsToKeep):
    new = dictionary.copy()
    for field in list(new.keys() - fieldsToKeep):
        new.pop(field, None)
    return new

new_Walls = list(map(lambda item: deleteFromDict(item, fieldsToKeep), allWalls))

{'id': 368080,
 'owner_id': -193013221,
 'date': 1613724008,
 'marked_as_ads': 0,
 'text': '📌 Коммуникационный центр Правительства Российской Федерации выпустил ежедневный отчет о текущей ситуации по борьбе с коронавирусом.\n\nСколько заразившихся в мире и России? Как проводится тестирование на коронавирус? Что делает Правительство для противодействия распространению инфекции? На все эти вопросы есть ответы в отчете.\n\n🔗 https://dalee.cdnvideo.ru/stopcoronavirus.rf/files/2021-02-19_coronavirus_government_report.pdf\n\nБольше новостей и оперативной информации о распространении коронавируса в России — на сайте стопкоронавирус.рф.\n\n#стопкоронавирус #здоровьевприоритете #coronavirus',
 'comments': {'count': 1, 'can_post': 1, 'groups_can_post': True},
 'likes': {'count': 15, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},
 'reposts': {'count': 5, 'user_reposted': 0},
 'views': {'count': 29408}}

In [67]:
type(new_Walls)

list

In [71]:
type(new_Walls[0])

dict

In [73]:
df = pd.DataFrame(new_Walls)
df

,id,owner_id,date,marked_as_ads,text,comments,likes,reposts,views
0,368080,-193013221,1613724008,0,📌 Коммуникационный центр Правительства Российс...,"{'count': 1, 'can_post': 1, 'groups_can_post':...","{'count': 15, 'user_likes': 0, 'can_like': 1, ...","{'count': 5, 'user_reposted': 0}",{'count': 29408}
1,368056,-193013221,1613722515,0,"По данным Роспотребнадзора, в мире по состояни...","{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 18, 'user_likes': 0, 'can_like': 1, ...","{'count': 4, 'user_reposted': 0}",{'count': 31141}
2,368044,-193013221,1613721700,0,За последние сутки в России выявлено 13 433 сл...,"{'count': 26, 'can_post': 1, 'groups_can_post'...","{'count': 61, 'user_likes': 0, 'can_like': 1, ...","{'count': 50, 'user_reposted': 0}",{'count': 17339}
3,367907,-193013221,1613664049,0,"🌿 Считается, что петрушка пришла к нам из Древ...","{'count': 5, 'can_post': 1, 'groups_can_post':...","{'count': 91, 'user_likes': 0, 'can_like': 1, ...","{'count': 51, 'user_reposted': 0}",{'count': 47681}
4,367876,-193013221,1613655029,0,На Дальнем Востоке постепенно ослабляют ограни...,"{'count': 3, 'can_post': 1, 'groups_can_post':...","{'count': 46, 'user_likes': 0, 'can_like': 1, ...","{'count': 14, 'user_reposted': 0}",{'count': 103637}
...,...,...,...,...,...,...,...,...,...
3483,13,-193013221,1584451200,0,Президент России Владимир Путин посетил коммун...,"{'count': 6, 'can_post': 1, 'groups_can_post':...","{'count': 16, 'user_likes': 0, 'can_like': 1, ...","{'count': 1, 'user_reposted': 0}",{'count': 8062}
3484,11,-193013221,1584447600,0,Не теряем полезную памятку❗\n\nМинздрав России...,"{'count': 40, 'can_post': 1, 'groups_can_post'...","{'count': 350, 'user_likes': 0, 'can_like': 1,...","{'count': 86, 'user_reposted': 0}",{'count': 28121}
3485,9,-193013221,1584440700,0,"Почему в России меньше заболеваний, чем в друг...","{'count': 10, 'can_post': 1, 'groups_can_post'...","{'count': 53, 'user_likes': 0, 'can_like': 1, ...","{'count': 8, 'user_reposted': 0}",{'count': 7219}
3486,5,-193013221,1584434734,0,Берегите себя и тщательно следите за своим здо...,"{'count': 235, 'can_post': 1, 'groups_can_post...","{'count': 1723, 'user_likes': 0, 'can_like': 1...","{'count': 344, 'user_reposted': 0}",{'count': 407326}


In [70]:
import pandas as pd
pd.read_json(new_Walls)

ValueError: Invalid file path or buffer object type: <class 'list'>

In [15]:
# list(map(lambda x: x * 2 + 3, numbers))
allWalls[0]

{'id': 368080,
 'owner_id': -193013221,
 'date': 1613724008,
 'marked_as_ads': 0,
 'text': '📌 Коммуникационный центр Правительства Российской Федерации выпустил ежедневный отчет о текущей ситуации по борьбе с коронавирусом.\n\nСколько заразившихся в мире и России? Как проводится тестирование на коронавирус? Что делает Правительство для противодействия распространению инфекции? На все эти вопросы есть ответы в отчете.\n\n🔗 https://dalee.cdnvideo.ru/stopcoronavirus.rf/files/2021-02-19_coronavirus_government_report.pdf\n\nБольше новостей и оперативной информации о распространении коронавируса в России — на сайте стопкоронавирус.рф.\n\n#стопкоронавирус #здоровьевприоритете #coronavirus',
 'comments': {'count': 1, 'can_post': 1, 'groups_can_post': True},
 'likes': {'count': 15, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},
 'reposts': {'count': 5, 'user_reposted': 0},
 'views': {'count': 29408}}

In [52]:
deleteFromDict(allWalls[2], fieldsToKeep)

donut
post_type
post_source
from_id
short_text_rate
is_favorite
carousel_offset
attachments


{'id': 368044,
 'owner_id': -193013221,
 'date': 1613721700,
 'marked_as_ads': 0,
 'text': 'За последние сутки в России выявлено 13 433 случая COVID-19 в 85 регионах, из них 1 462 (10,9%) — активно у контактных лиц без клинических проявлений болезни.\n\nВ Российской Федерации нарастающим итогом зарегистрирован 4 139 031 случай (+0,33%) коронавирусной инфекции в 85 регионах, число выписанных пациентов выросло до 3 679 949.\n\nЗа прошедшие сутки в связи с выздоровлением выписано 18 637 человек. Больше всего выписано:\n🔹 Санкт-Петербург – 3 676\n🔹 Москва – 2 112\n🔹 Московская область – 598\n\nЗа последние сутки подтверждено 470 летальных исходов с коронавирусом:\n🔻 Москва – 53\n🔻 Санкт-Петербург – 47\n🔻 Московская область – 35\n🔻 Ростовская область – 23\n🔻 Краснодарский край – 16\n🔻 Красноярский край – 14\n🔻 Свердловская область – 13\n🔻 Нижегородская область – 12\n🔻 Воронежская область – 11\n🔻 Ленинградская область – 11\n🔻 Пермский край – 9\n🔻 Алтайский край – 9\n🔻 Владимирская область – 

In [54]:
new_Walls = list(map(lambda item: deleteFromDict(item, fieldsToKeep), allWalls))

In [57]:
new_Walls[4]

{'id': 367876,
 'owner_id': -193013221,
 'date': 1613655029,
 'marked_as_ads': 0,
 'text': 'На Дальнем Востоке постепенно ослабляют ограничения, связанные с коронавирусом.\n\n🧓 В Хабаровском крае сняли режим самоизоляции для пожилых людей. Ограничения сохранили только для Аяно-Майского района, где наблюдается максимальное количество заражений.\n\n👩\u200d🍳 К обычному режиму работы в регионе уже вернулись ночные клубы, кафе и рестораны. Ограничения в социальных учреждениях планируется снять в марте.\n\n💪🏻 Не отстает и Чукотский автономный округ. В регионе ослабили ограничения для привитых от коронавируса. Местные спортсмены теперь могут участвовать в соревнованиях в закрытых помещениях, а пожилые люди - посещать досуговые мероприятия.\n\n😷 Тем не менее, в обоих регионах сохраняется масочный режим в общественных местах и требования к соблюдению социальной дистанции.\n\n💉 Всего в Хабаровском крае с начала пандемии зарегистрировано 47,4 тыс. заболевших. В регионе активно идет вакцинация: на

In [362]:
from datetime import datetime
ts = int("1616513397")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2021-03-23 18:29:57
